In [1]:
import sys
sys.path.append('/home/yanrui/code/CLIPBased_TAD')

In [2]:
import os 
os.environ['CUDA_VISIBLE_DEVICES']='0' 

In [3]:
from model.models import RFCLIP
# import embedding
from model.CLIPTAD.CLIPModel import CLIPModel
from model.XRF.embedding import TADEmbedding_pure
from model.TAD.embedding import Embedding
# fusion
from model.fusion import GatedFusionAdd2, GatedFusionWeight
# import backbone
from model.TAD.tad_backbone import TADBackbone
from configs.config import config

/home/yanrui/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch
B = 2
C1 = 270
C2 = 30
L = 2048
wifi = torch.rand([B, C1, L])
imu = torch.rand([B, C2, L])

wifi.shape

torch.Size([2, 270, 2048])

In [5]:
cnn_embedding_wifi = Embedding(270)
cnn_embedding_imu = Embedding(30)
add_fusion = GatedFusionAdd2(512)
weight_fusion = GatedFusionWeight(512)


CLIPEmbed = CLIPModel(config)
clip_embedding_wifi = CLIPEmbed.embedding_wifi
clip_embedding_imu = CLIPEmbed.embedding_imu
clip_fusion = CLIPEmbed.fusion # B * 512 * L

In [6]:
cnn_wifi = cnn_embedding_wifi(wifi)
cnn_imu = cnn_embedding_imu(imu)

cnn_wifi.shape, cnn_imu.shape

(torch.Size([2, 512, 2048]), torch.Size([2, 512, 2048]))

In [14]:
from model.XRF.fusion import GatedFusionWeight, GatedFusion, GatedFusionAdd2

add_fusion = GatedFusionAdd2(512, isTrainClip=True)


In [15]:


add_fusion_res = add_fusion(cnn_wifi, cnn_imu)


In [16]:
add_fusion_res.shape

torch.Size([2, 2048, 512])

In [12]:
clip_wifi = clip_embedding_wifi(wifi)
clip_imu = clip_embedding_imu(imu)

In [13]:
clip_wifi.shape, clip_imu.shape

(torch.Size([2, 512, 2048]), torch.Size([2, 512, 2048]))

In [14]:
from model.XRF.embedding import TADEmbedding_pure
tad_emb = TADEmbedding_pure(512)
xxx = tad_emb(add_fusion_res)
xxx.shape

torch.Size([2, 512, 256])

# test mamba

In [ ]:
from mamba_ssm.modules.mamba_simple_vim import Mamba as ViM
from mamba_ssm.modules.mamba_simple_bim import Mamba as DBM

mamba = DBM(10, d_conv=2, use_fast_path=True, expand=1)
mamba = ViM(10, d_conv=2, bimamba_type="v2", use_fast_path=True)

## go on test mamba

In [2]:
import sys
sys.path.append('/home/yanrui/code/CLIPBased_TAD')

from model.XRF.mamba_backbone import Mamba, Mamba2


from configs.config import config

In [3]:
mamba = Mamba2(config).to('cuda')

In [4]:
import torch
embedd = torch.rand([2, 512, 256]).to('cuda')
embedd.shape

torch.Size([2, 512, 256])

In [5]:
global_feat = torch.rand([2, 512, 2048]).to('cuda')

In [6]:
feats = mamba(global_feat, embedd)


In [9]:
feats[2].shape

torch.Size([2, 512, 32])

## test mymamba